# Spark, v2

Massive attempt to run spark with jar files on current node in local mode, but no luck.

You can skip this notebook.

## Implementation


### IMPORTANT NOTE!

Due to very ppor machine I have, I have to read & process HOURLY packages instead of running spark for the whole day of even bigger period.

### Impl

In [1]:
import pathlib, os

In [2]:
spark_resources_path = "/home/jovyan/work/data/"
spark_resources_path_obj = pathlib.Path(spark_resources_path)

In [3]:
import pyspark

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [13]:
# master = "spark://spark:7077"
# conf = SparkConf().setAppName("Spark TEST v2") #.setMaster(master)

In [5]:
# jars_to_load = [
#     "clickhouse-jdbc-0.4.6-all.jar",
#     "clickhouse-spark-runtime-3.3_2.12-0.7.2.jar",
#     "clickhouse-native-jdbc-shaded-2.5.4.jar"
# ]

# jar_path_joined = ":".join(["/opt/spark/resources/jars/{}".format(jar_name) for jar_name in jars_to_load])
# jar_path_joined

'/opt/spark/resources/jars/clickhouse-jdbc-0.4.6-all.jar:/opt/spark/resources/jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar:/opt/spark/resources/jars/clickhouse-native-jdbc-shaded-2.5.4.jar'

In [6]:
# os.environ['PYSPARK_SUBMIT_ARGS'] = \
#   '--jars {} pyspark-shell'.format(":".join(["../jars/{}".format(jar_name) for jar_name in jars_to_load]))


In [7]:
# conf.set('spark.jars', jar_path_joined)
# conf.set('spark.driver.extraClassPath', jar_path_joined)
# conf.set('spark.driver.extraLibraryPath', jar_path_joined)

In [8]:
# conf.getAll()

[('spark.app.name', 'Spark TEST v2'),
 ('spark.master', 'spark://spark:7077'),
 ('spark.jars',
  '/opt/spark/resources/jars/clickhouse-jdbc-0.4.6-all.jar:/opt/spark/resources/jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar:/opt/spark/resources/jars/clickhouse-native-jdbc-shaded-2.5.4.jar'),
 ('spark.driver.extraClassPath',
  '/opt/spark/resources/jars/clickhouse-jdbc-0.4.6-all.jar:/opt/spark/resources/jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar:/opt/spark/resources/jars/clickhouse-native-jdbc-shaded-2.5.4.jar'),
 ('spark.driver.extraLibraryPath',
  '/opt/spark/resources/jars/clickhouse-jdbc-0.4.6-all.jar:/opt/spark/resources/jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar:/opt/spark/resources/jars/clickhouse-native-jdbc-shaded-2.5.4.jar')]

In [4]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
spark.sparkContext

<SparkContext master=local[*] appName=Python Spark SQL basic example>

In [6]:
print(spark.sparkContext._jsc.sc().listJars())

Vector()


In [7]:
given_date = "2023-07-19"
glob_pattern = "{}-*.json.gz".format(given_date)

In [8]:
paths = (str(path) for path in sorted(spark_resources_path_obj.glob(glob_pattern), key=os.path.getmtime))
cur_path = next(paths)
print(cur_path)

/home/jovyan/work/data/2023-07-19-0.json.gz


In [9]:
df = spark.read.json(cur_path)

In [10]:
df.count()

129321

In [11]:
df.createOrReplaceTempView("activity")

In [12]:
spark.sql(
    "SELECT min(created_at), max(created_at) FROM activity"
).show(10)

+--------------------+--------------------+
|     min(created_at)|     max(created_at)|
+--------------------+--------------------+
|2023-07-19T00:00:00Z|2023-07-19T00:59:59Z|
+--------------------+--------------------+



In [13]:
spark.sql(
    "SELECT type, count(*) FROM activity group by type order by count(*) desc"
).show(50)

+--------------------+--------+
|                type|count(1)|
+--------------------+--------+
|           PushEvent|   72942|
|         CreateEvent|   17464|
|    PullRequestEvent|   12203|
|   IssueCommentEvent|    5995|
|          WatchEvent|    5709|
|         DeleteEvent|    4608|
|PullRequestReview...|    2975|
|         IssuesEvent|    2363|
|PullRequestReview...|    1559|
|           ForkEvent|    1426|
|        ReleaseEvent|     680|
|  CommitCommentEvent|     561|
|         PublicEvent|     406|
|         MemberEvent|     270|
|         GollumEvent|     160|
+--------------------+--------+



## Metrics

* List of Developers that own more than one repository;
* List of Developers who did more than one commit in a day, ordered by name and number of commits;
* List of Developers with less than one commit in a day;
* Total Developers grouped by gender;
* Total projects with more than 10 members;

### LoD - Own > 1 Repo

In [14]:
split_col = pyspark.sql.functions.split(df['repo.name'], '/')

df = df.withColumn('repo_author', split_col.getItem(0))
df = df.withColumn('repo_name', split_col.getItem(1))

In [15]:
df.createOrReplaceTempView("activity")

Provide next list of __DISTINCT__ records:
```
| date | hour | repo author | repo_name |
```

In [16]:
df_repos = spark.sql(
    """
    SELECT
        date(created_at) as date,
        hour(created_at) as hour,
        repo_author, 
        repo_name,
        count(*) as total_events
    FROM activity 
    GROUP BY 
        date,
        hour,
        repo_author,
        repo_name
    ORDER BY total_events DESC
    """
)

In [17]:
df_repos.show()

+----------+----+-----------------+--------------------+------------+
|      date|hour|      repo_author|           repo_name|total_events|
+----------+----+-----------------+--------------------+------------+
|2023-07-19|   0|         joeIvan2|       btcOrderIndex|        1250|
|2023-07-19|   0|         sgou1969|   streamlit-example|         927|
|2023-07-19|   0|    happyfish2024|                mins|         712|
|2023-07-19|   0|        B74LABgit|                 CAM|         581|
|2023-07-19|   0|  bullet-dev-team|   demo-app-env-list|         521|
|2023-07-19|   0|  bullet-dev-team|python-pyramid-pu...|         512|
|2023-07-19|   0|       CyberCommy|biqu520net-30001-...|         348|
|2023-07-19|   0|appseed-projects3|6d11e92f-b79e-423...|         348|
|2023-07-19|   0|          unifyai|                 ivy|         339|
|2023-07-19|   0|      hacks-guide|           Guide_3DS|         338|
|2023-07-19|   0| liangbin-foxmail|             GONGKAI|         262|
|2023-07-19|   0|  n

In [19]:
def compresion_degree(df_new, df_old):
    print(
        """
        Degree of ~compression~ via aggregation: {:0.2f} ({} / {})
        """.format(df_old.count() / df_new.count(), df_new.count(), df_old.count())
    )

In [20]:
compresion_degree(df_repos, df)


        Degree of ~compression~ via aggregation: 2.59 (49955 / 129321)
        


#### Limitations & Problems

1. As the snapshots in GHARCHIVE are hourly-based, there is __no data__ about the past activity for the repositories. \
So without the past activity, the number of owned repos is incomplete and can be based only on the activities, that we've crawled & stored. \
On how to overcome this, please read the next section, `How can it be improved?`

#### How can it be improved?

* Get bigger slice of data - and do the next thing:
  * Store daily snapshots of all unique combinations of `date`, `hour`, `repo.author`, `repo.name`
  * Provide the metrics by getting the global `count(distinct repo.name)` aggregation on group of `repo.author`-s

#### Loading - Clickhouse

In [21]:
import clickhouse_connect
import os

client = clickhouse_connect.get_client(
    host='clickhouse_server', 
    username=os.environ['CLICKHOUSE_USER'], 
    password=os.environ['CLICKHOUSE_PASSWORD']
)

In [22]:
client.command('USE gharchive;')

''

In [23]:
client.command("show tables;")

''

In [24]:
create_table_cmd = """
CREATE TABLE IF NOT EXISTS gharchive.repo_aggregated
(
    date Date,
    hour UInt8,
    repo_author String,
    repo_name String,
    total_events UInt32
)
ENGINE MergeTree
ORDER BY date;
"""

print(create_table_cmd)

client.command(create_table_cmd)


CREATE TABLE IF NOT EXISTS gharchive.repo_aggregated
(
    date Date,
    hour UInt8,
    repo_author String,
    repo_name String,
    total_events UInt32
)
ENGINE MergeTree
ORDER BY date;



''

In [30]:
create_table_cmd = """
select count(*) from gharchive.repo_aggregated

"""

print(create_table_cmd)

client.command(create_table_cmd)


select count(*) from gharchive.repo_aggregated




7733645

In [25]:
# spark.sparkContext.addFile("../jars/clickhouse-jdbc-0.4.6-all.jar")

# spark.sparkContext._jsc.addJar("../jars/clickhouse-jdbc-0.4.6-all.jar")

# spark.sparkContext.addFile("../jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar")

# spark.sparkContext._jsc.addJar("../jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar")

# spark.sparkContext

In [26]:
# spark.sparkContext.addFile("/home/jovyan/work/jars/clickhouse-jdbc-0.4.6-all.jar")

# spark.sparkContext._jsc.addJar("/home/jovyan/work/jars/clickhouse-jdbc-0.4.6-all.jar")

# spark.sparkContext.addFile("/home/jovyan/work/jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar")

# spark.sparkContext._jsc.addJar("/home/jovyan/work/jars/clickhouse-spark-runtime-3.3_2.12-0.7.2.jar")

# spark.sparkContext

In [27]:
print(spark.sparkContext._jsc.sc().listJars())

Vector()


In [28]:
print(spark.sparkContext._jsc.sc().listJars())

Vector()


In [29]:
type(df_repos)

pyspark.sql.dataframe.DataFrame

In [31]:
# df_repos.read \
#     .format("jdbc") \
#     .mode("append") \
#     .option("driver", "com.github.housepower.jdbc.ClickHouseDriver") \
#     .option("url", "jdbc:clickhouse://clickhouse-server:9000/gharchive") \
#     .option("user", os.environ['CLICKHOUSE_USER']) \
#     .option("password", os.environ['CLICKHOUSE_PASSWQRD']) \
#     .option("dbtable", "gharchive.repo_aggregated") \
#     .load()

In [32]:
# df_repos.write \
#     .format("jdbc") \
#     .mode("append") \
#     .option("driver", "com.github.housepower.jdbc.ClickHouseDriver") \
#     .option("url", "jdbc:clickhouse://clickhouse-server:9000/gharchive") \
#     .option("user", os.environ['CLICKHOUSE_USER']) \
#     .option("password", os.environ['CLICKHOUSE_PASSWQRD']) \
#     .option("dbtable", "gharchive.repo_aggregated") \
#     .save()

### LoD - Did > 1 Commit a day

https://stackoverflow.com/questions/64605066/explode-array-with-nested-array-raw-spark-sql

#### Notes

1. Commits are present inside `payload.commits.author.name`.\
Given the hourly and typized nature of the dataset, we can:
    * Filter by `type`, leaving only `PushEvent`-s
    * Explode author names
    * Get the count aggregation

#### Impl

In [44]:
df.select(
    df["payload.commits.author"],
    df["payload.commits.author.name"],
    df["payload.ref"],
    df["payload.ref_type"],
    df["payload.review"],
    df["type"],
).show(50)

+--------------------+--------------------+--------------------+----------+------+--------------------+
|              author|                name|                 ref|  ref_type|review|                type|
+--------------------+--------------------+--------------------+----------+------+--------------------+
|                null|                null|                null|      null|  null|    PullRequestEvent|
|                null|                null|                null|repository|  null|         CreateEvent|
|[{cottaris@upenn....|  [Nicolas Cottaris]|refs/heads/Mosaic...|      null|  null|           PushEvent|
|[{87268091+DXPGH@...|           [Danny P]|     refs/heads/main|      null|  null|           PushEvent|
|[{57704129+soundo...|       [soundofhope]|   refs/heads/master|      null|  null|           PushEvent|
|                null|                null|                null|      null|  null|    PullRequestEvent|
|                null|                null|                null|

In [43]:
df_commits = spark.sql(
    """
    SELECT
        date(created_at) as date,
        created_at,
        explode(payload.commits.author.name) as author_name
    FROM activity
    WHERE 
        type = 'PushEvent'
    """
)

df_commits.show(10)

compresion_degree(df_commits, df)

+----------+--------------------+----------------+
|      date|          created_at|     author_name|
+----------+--------------------+----------------+
|2023-07-19|2023-07-19T00:00:00Z|Nicolas Cottaris|
|2023-07-19|2023-07-19T00:00:00Z|         Danny P|
|2023-07-19|2023-07-19T00:00:00Z|     soundofhope|
|2023-07-19|2023-07-19T00:00:00Z|          server|
|2023-07-19|2023-07-19T00:00:00Z|        19920513|
|2023-07-19|2023-07-19T00:00:00Z|          Jaison|
|2023-07-19|2023-07-19T00:00:00Z|         gabriel|
|2023-07-19|2023-07-19T00:00:00Z|     mtssantos96|
|2023-07-19|2023-07-19T00:00:00Z|     fanmingming|
|2023-07-19|2023-07-19T00:00:00Z|     fanmingming|
+----------+--------------------+----------------+
only showing top 10 rows


        Degree of ~compression~ via aggregation: 1.13 (114408 / 129321)
        


In [36]:
spark.sql(
    """
    SELECT 
        sq.created_at_date as date,
        sq.author_name, 
        count(*) total_commits from 
    (
        SELECT
            date(created_at) as created_at_date,
            explode(payload.commits.author.name) as author_name
        FROM activity
        WHERE 
            type = 'PushEvent'
    ) sq
    GROUP BY 
        date, 
        sq.author_name
    HAVING total_commits > 1
    ORDER BY 
        total_commits DESC, 
        author_name DESC
    """
).show(10)

+----------+-------------------+-------------+
|      date|        author_name|total_commits|
+----------+-------------------+-------------+
|2023-07-19|github-actions[bot]|         5780|
|2023-07-19|        Upptime Bot|         2603|
|2023-07-19|      renovate[bot]|         1927|
|2023-07-19|    dependabot[bot]|         1600|
|2023-07-19|           joeIvan2|         1250|
|2023-07-19|           sgou1969|          927|
|2023-07-19|      happyfish2024|          712|
|2023-07-19|          B74LABgit|          707|
|2023-07-19|        Yifei Zhang|          674|
|2023-07-19|     github-actions|          667|
+----------+-------------------+-------------+
only showing top 10 rows



#### Limitations & Problems

1. Potentially, subquery usage in Spark is not a good practice. Yet, it may be used for prototype reasons.
2. We have to additionally group by date, because we have to provide DAILY number of commits.

#### How can it be improved?

* Probably, we should get rid of bot entities :)

### [!!!] LoD - < 1 commit in a day

#### Notes

* The trick is that an author can have 1 commit in, let's say, 25h time span, which is more than 1 day. \
So we should regroup data not by a date, but by consecutive dataspansm which are 25h each...
* For to test it, we need at least __2 days worth of data__! \
For now I have complications to get this data, so I skip the metric.

#### Impl

In [73]:
spark.sql(
    """
    SELECT 
        sq.created_at_date as date,
        sq.author_name, 
        count(*) total_commits from 
    (
        SELECT
            date(created_at) as created_at_date,
            explode(payload.commits.author.name) as author_name
        FROM activity
        WHERE 
            type = 'PushEvent'
    ) sq
    GROUP BY 
        date, 
        sq.author_name
    HAVING total_commits <= 1
    ORDER BY 
        total_commits DESC, 
        author_name DESC
    """
).show(10)

+----------+--------------------+-------------+
|      date|         author_name|total_commits|
+----------+--------------------+-------------+
|2023-07-28|            🤖 R2-D2|            1|
|2023-07-28|      💵 moneybot 💵|            1|
|2023-07-28| 🐼 Samrose Ahmed 🐼|            1|
|2023-07-27|𒀳 Scribe of the ...|            1|
|2023-07-27|                  ｚ|            1|
|2023-07-28|              황지환|            1|
|2023-07-28|              홍세빈|            1|
|2023-07-27|              현기홍|            1|
|2023-07-28|                현경|            1|
|2023-07-27|              한창수|            1|
+----------+--------------------+-------------+
only showing top 10 rows



### [!!!] Total Developers grouped by gender

#### Notes

* Proabably can be done via `user[profile_pronouns]` , but for EACH unique GitHub User... That's just dumb.
* I haven't found such info (sex,gender, pronouns) in the dataset.

### Total projects with more than 10 members

#### Notes

* Where to find members ???

#### IMPL


In [38]:
spark.sql(
    """
    SELECT
        type,
        payload.member
    FROM activity
    WHERE
        payload.member is not NULL
        
    """
).show(10)

+-----------+--------------------+
|       type|              member|
+-----------+--------------------+
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
+-----------+--------------------+
only showing top 10 rows



In [40]:
df.select(
    df['created_at'],
    df["payload.member"],
    df["payload.member.id"],
    df["payload.member.login"],
    df["payload.member.type"],
    df["payload.action"],
    df["org.login"],
    df["actor.login"],
    df["repo.name"],
    df["type"],
).filter(df["type"] == 'MemberEvent').show(50)

+--------------------+--------------------+---------+-----------------+----+------+------------------+------------------+--------------------+-----------+
|          created_at|              member|       id|            login|type|action|             login|             login|                name|       type|
+--------------------+--------------------+---------+-----------------+----+------+------------------+------------------+--------------------+-----------+
|2023-07-19T00:00:25Z|{https://avatars....| 88982841|           MNRMax|User| added|          SammyGPT|          AntoDono|SammyGPT/SammyGPT...|MemberEvent|
|2023-07-19T00:00:31Z|{https://avatars....| 85177319|       lavywei666|User| added|              null|      Alexstarwing|Alexstarwing/Hire...|MemberEvent|
|2023-07-19T00:00:37Z|{https://avatars....|130781666|          Gellart|User| added|   Hackademy-PT-08|   Hackademy-PT-08|Hackademy-PT-08/l...|MemberEvent|
|2023-07-19T00:00:51Z|{https://avatars....|117898665|     PierColina98

In [62]:
df_members = spark.sql(
    """
    SELECT
        date(created_at) as date,
        timestamp(created_at) as created_at,
        payload.member.login as member_login,
        payload.member.type as member_type,
        org.login as org_login,
        actor.login as actor_login,
        repo.name as repo_name_full,
        repo_name,
        repo_author
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    """
)

df_members.show(10)

compresion_degree(df_members, df)

+----------+-------------------+--------------+-----------+---------------+---------------+--------------------+--------------------+---------------+
|      date|         created_at|  member_login|member_type|      org_login|    actor_login|      repo_name_full|           repo_name|    repo_author|
+----------+-------------------+--------------+-----------+---------------+---------------+--------------------+--------------------+---------------+
|2023-07-19|2023-07-19 00:00:25|        MNRMax|       User|       SammyGPT|       AntoDono|SammyGPT/SammyGPT...|   SammyGPT-Frontend|       SammyGPT|
|2023-07-19|2023-07-19 00:00:31|    lavywei666|       User|           null|   Alexstarwing|Alexstarwing/Hire...|            Hirebeat|   Alexstarwing|
|2023-07-19|2023-07-19 00:00:37|       Gellart|       User|Hackademy-PT-08|Hackademy-PT-08|Hackademy-PT-08/l...|laravel_04_Gellar...|Hackademy-PT-08|
|2023-07-19|2023-07-19 00:00:51|  PierColina98|       User|           null|KevinVargas1999|KevinVarg

In [63]:
df_members.dtypes

[('date', 'date'),
 ('created_at', 'timestamp'),
 ('member_login', 'string'),
 ('member_type', 'string'),
 ('org_login', 'string'),
 ('actor_login', 'string'),
 ('repo_name_full', 'string'),
 ('repo_name', 'string'),
 ('repo_author', 'string')]

In [118]:
spark.sql(
    """
    SELECT
        repo.name,
        count(distinct payload.member.login) as total_member_logins
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    GROUP BY
        repo.name
    ORDER BY
        total_member_logins DESC
    """
).show(10)

+--------------------+-------------------+
|                name|total_member_logins|
+--------------------+-------------------+
|jgranadoscunoc/re...|                 29|
|Jucer74/WebDevelo...|                 17|
|hdtoledo/nodejs_a...|                 11|
|InstrucJavaReclui...|                  9|
|ChungLeba/nextjsv...|                  9|
|iramgutierrez/433...|                  9|
|emrchi/TestNGProj...|                  8|
|aydaakcay/TestPro...|                  7|
|ufuk-muhsiroglu/T...|                  7|
|GoldenMEmre/com.w...|                  7|
+--------------------+-------------------+
only showing top 10 rows



In [119]:
spark.sql(
    """
    SELECT
        date(created_at) as date,
        hour(created_at) as hour,
        repo.name,
        count(distinct payload.member.login) as total_member_logins
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    GROUP BY
        date, hour, repo.name
    ORDER BY
        total_member_logins DESC
    """
).show(10)

+----------+----+--------------------+-------------------+
|      date|hour|                name|total_member_logins|
+----------+----+--------------------+-------------------+
|2023-07-27|  20|jgranadoscunoc/re...|                 21|
|2023-07-28|   1|Jucer74/WebDevelo...|                 16|
|2023-07-28|   1|InstrucJavaReclui...|                  9|
|2023-07-27|  19|hdtoledo/nodejs_a...|                  9|
|2023-07-27|  19|emrchi/TestNGProj...|                  7|
|2023-07-27|  19|ufuk-muhsiroglu/T...|                  7|
|2023-07-27|  18|aydaakcay/TestPro...|                  7|
|2023-07-27|  18|GoldenMEmre/com.w...|                  7|
|2023-07-27|  19|mehmetfilik/comWo...|                  6|
|2023-07-27|  19|FabianoCarneiro/p...|                  6|
+----------+----+--------------------+-------------------+
only showing top 10 rows



In [122]:
spark.sql(
    """
    SELECT
        date(created_at) as date,
        hour(created_at) as hour,
        created_at,
        repo.name,
        payload.member.login as member_name
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    ORDER BY
        date ASC,
        hour ASC,
        created_at ASC,
        repo.name ASC,
        member_name ASC
    """
).show(10)

+----------+----+--------------------+--------------------+---------------+
|      date|hour|          created_at|                name|    member_name|
+----------+----+--------------------+--------------------+---------------+
|2023-07-27|  18|2023-07-27T18:00:15Z|noirblancrouge/Pa...|        bsozeau|
|2023-07-27|  18|2023-07-27T18:00:31Z| Spinofficial/printf|     RolivhuwaN|
|2023-07-27|  18|2023-07-27T18:00:33Z| faellm/calculoRotas|    stheeCamile|
|2023-07-27|  18|2023-07-27T18:00:39Z|Programmer231/Roc...|   bevin-crypto|
|2023-07-27|  18|2023-07-27T18:00:41Z|    Drefdu/DentalNew|AntonioEstrada0|
|2023-07-27|  18|2023-07-27T18:00:44Z|code-X16/simple_s...|      udeme-goc|
|2023-07-27|  18|2023-07-27T18:01:13Z|maxDes23/project-...|       MadDog83|
|2023-07-27|  18|2023-07-27T18:01:15Z|Diego582/ideas-pi...|       FedeSabi|
|2023-07-27|  18|2023-07-27T18:01:16Z|CaravanStudios/dc...| caseylivingood|
|2023-07-27|  18|2023-07-27T18:01:20Z|devfabien/Gym-git...| IbrahimBagalwa|
+----------+

#### Limitations & Problems

1. As the snapshots in GHARCHIVE are hourly-based, there is __no data__ about the past activity for the repositories. \
So without the past activity, the number of members is incomplete and can be based only on the activities, like 'MemberEvent', that we've crawled & stored. \
On how to overcome this, please read the next section, `How can it be improved?`

#### How can it be improved?

* Get bigger slice of data - and do the next thing:
  * Store daily snapshots of all unique combinations of `date`, `repo`, `member`, 'action'
  * Provide the metrics by getting the global `count(distinct member)` aggregation on group of `repo`-s